In [ ]:
from pycaret.regression import *
import pandas as pd
import numpy as np
df = pd.read_csv(r'https://raw.githubusercontent.com/data-to-insight/ERN-sessions/main/data/1980%202023%20average%20house%20prices.csv')
df['Period'] = pd.to_datetime(df['Period'], format="%Y-%m")
 
df =df[['Period', 'Average price All property types']]
df

In [ ]:
# model_data = df[df['Period'] < pd.to_datetime("2011", format="%Y")]
# real_future_data = df[df['Period'] >= pd.to_datetime("2011", format="%Y")]
# real_future_data
model_data = df[df['Period'] > pd.to_datetime("2002", format="%Y")]
model_data = model_data[model_data['Period'] <= pd.to_datetime("2008", format="%Y")]
real_future_data = df[df['Period'] >= pd.to_datetime("2008", format="%Y")]
 
model_data

In [ ]:
# date - dates
# ordinal - scales/rankings
# catergorical - categories
# numberic - continuous
s =setup(data=model_data, 
         train_size=0.7,
         target= 'Average price All property types',
         date_features=['Period'],
         fold=2,
         fold_shuffle=False,
         data_split_shuffle=False,
         fold_strategy='timeseries',
         session_id=123
         )


In [ ]:
best = compare_models(sort='MAE')

In [ ]:
plot_model(best, plot='error')

In [ ]:
future_dates = pd.date_range(start= '2011-01-01', end='2030-01-01', freq='MS')
future_df = pd.DataFrame()
future_df['Period'] = future_dates
predictions_future = predict_model(best, data=future_df)
predictions_future.head(20)

In [ ]:
import plotly.express as px
concat_df = pd.concat([model_data, predictions_future], axis=0)
fig = px.line(concat_df, x=concat_df['Period'], y=['Average price All property types', 'prediction_label'])
fig.add_scatter(x=df['Period'], y=df['Average price All property types'])

In [ ]:
final_val = model_data['Average price All property types'].iloc[-1]
first_val = predictions_future['prediction_label'].iloc[0]
predictions_future['corrected val'] = predictions_future['prediction_label'] - first_val + final_val
concat_df = pd.concat([model_data, predictions_future], axis=0)
fig = px.line(concat_df, x=concat_df['Period'], y=['Average price All property types', 'prediction_label'])
fig.add_scatter(x=df['Period'], y=df['Average price All property types'])